In [0]:
%sql

create database if not exists demo_db


In [0]:
%sql
create table if not exists demo_db.fire_service_calls_tbl(
  CallNumber integer,
  UnitID string,
  IncidentNumber integer,
  CallType string,
  CallDate string,
  WatchDate string,
  CallFinalDisposition string,
  AvailableDtTm string,
  Address string,
  City string,
  Zipcode integer,
  Battalion string,
  StationArea string,
  Box string,
  OriginalPriority string,
  Priority string,
  FinalPriority integer,
  ALSUnit boolean,
  CallTypeGroup string,
  NumAlarms integer,
  UnitType string,
  UnitSequenceInCallDispatch integer,
  FirePreventionDistrict string,
  SupervisorDistrict string,
  Neighborhood string,
  Location string,
  RowID string,
  Delay float
) using delta

In [0]:
%sql
INSERT INTO demo_db.fire_service_calls_tbl VALUES (
  1001,                            -- CallNumber
  'E231',                         -- UnitID
  200001,                         -- IncidentNumber
  'Medical Incident',            -- CallType
  '2023-07-01',                  -- CallDate
  '2023-07-01',                  -- WatchDate
  'Closed',                      -- CallFinalDisposition
  '2023-07-01T14:45:00',         -- AvailableDtTm
  '123 Main St',                 -- Address
  'San Francisco',               -- City
  94102,                         -- Zipcode
  'B01',                         -- Battalion
  'S01',                         -- StationArea
  '1234',                        -- Box
  '3',                           -- OriginalPriority
  '3',                           -- Priority
  3,                             -- FinalPriority
  TRUE,                          -- ALSUnit
  'Medical',                     -- CallTypeGroup
  1,                             -- NumAlarms
  'Engine',                      -- UnitType
  1,                             -- UnitSequenceInCallDispatch
  'FPD1',                        -- FirePreventionDistrict
  'SD1',                         -- SupervisorDistrict
  'Downtown',                    -- Neighborhood
  '(37.7749, -122.4194)',        -- Location
  'abc123',                      -- RowID
  5.2                            -- Delay
);


In [0]:
%sql
select * from demo_db.fire_service_calls_tbl

In [0]:
%sql
DELETE FROM demo_db.fire_service_calls_tbl
WHERE CallNumber = 1001;


In [0]:
%sql
SELECT * FROM demo_db.fire_service_calls_tbl
WHERE CallNumber = 1001;


In [0]:
%sql
select * from fire_service_calls_view

In [0]:
fire_df = spark.read.format("csv")\
      .option("header",True)\
          .option("inferSchema",True)\
              .load("/Volumes/udemypyspark/default/firstvolume/fire-calls.csv")

In [0]:
spark.sql("DROP TABLE IF EXISTS demo_db.fire_service_calls_tbl")


In [0]:
fire_df.write.format("delta").mode("overwrite").saveAsTable("demo_db.fire_service_calls_tbl")

In [0]:
from pyspark.sql.functions import col

# Rename columns with invalid characters
new_cols = [col(c).alias(c.replace(" ", "_")) for c in fire_df.columns]
fire_df_cleaned = fire_df.select(*new_cols)

# Save to Delta table
fire_df_cleaned.write.format("delta").mode("overwrite").saveAsTable("demo_db.fire_service_calls_tbl")


In [0]:
fire_df_cleaned.write.format("delta").mode("overwrite").saveAsTable("demo_db.fire_service_calls_tbl")

In [0]:
df = spark.sql("SELECT * FROM demo_db.fire_service_calls_tbl")
df.show()


In [0]:
display(df)

In [0]:
%sql
DESCRIBE FORMATTED demo_db.fire_service_calls_tbl


In [0]:
%sql
SELECT * FROM demo_db.fire_service_calls_tbl

In [0]:
%sql
select count(*) from demo_db.fire_service_calls_tbl

### How many distinct types of calls were made to the fire department?

In [0]:
%sql
select count(distinct Call_Type) asdistinct_call_type_count
from demo_db.fire_service_calls_tbl where Call_Type is not null

### what were distinct types of calls made to fire department

In [0]:
%sql
select distinct Call_Type from demo_db.fire_service_calls_tbl where Call_type is not null

### what were the most common call types?

In [0]:
%sql
select Call_Type,count(*) as count
from demo_db.fire_service_calls_tbl
where Call_Type is not null
group by Call_Type
order by count desc

### what zip codes accounted for most common calls?

In [0]:
%sql
select Call_Type,Zipcode_of_Incident,count(*) as count
from demo_db.fire_service_calls_tbl
where Call_Type is not null
group by Call_Type,Zipcode_of_Incident
order by count desc

### How many distinct years of data is in the dataset?

In [0]:
%sql
select distinct(year(to_date(Call_Date,'MM/dd/yyyy'))) as year_num
from demo_db.fire_service_calls_tbl
order by year_num desc

### what week of year in 2025 had most fire calls?

In [0]:
%sql
select weekofyear(to_date(Call_Date,'MM/dd/yyyy')) as week_num,count(*) as count
from demo_db.fire_service_calls_tbl
where year(to_date(Call_Date,'MM/dd/yyyy')) = 2025
group by week_num
order by count desc

In [0]:
fire_df = spark.read.format("csv")\
      .option("header",True)\
          .option("inferSchema",True)\
              .load("/Volumes/udemypyspark/default/firstvolume/fire-calls.csv")

In [0]:
display(fire_df)